In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch09. 9장 chroma→pinecone을 활용한 RAG 구현(upstage) </font></b>

# 벡터DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    - https://www.pinecone.io 에서 api key 생성 → .env에 추가(PINECONE_API_KEY 등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [2]:
# embedding : upstage의 solar-embedding-1-large-passage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

In [4]:
len(embedding.embed_query('소득세법'))

4096

In [5]:
%%time

# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# 업로드시 시간이 오래걸려서 경고가 뜨는데 안보이려면 아나콘다 프롬프트 llm 환경에서 conda install -c conda-forge ipywidgets

# 데이터를 처음 업로드할 때
index_name = 'tax-index-upstage'
# database = PineconeVectorStore.from_documents(documents=document_list,
#                                               embedding=embedding,
#                                               index_name=index_name)

# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
                               index_name=index_name)

CPU times: total: 7.36 s
Wall time: 38.6 s


# 2. 답변 생성을 위한 Retrieval

In [6]:
query = '연봉이 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # 기본 k 값 : 4

In [7]:
# retrieved_docs[0].page_content

retrieved_doc = '\n\n---\n\n'.join([doc.page_content for doc in retrieved_docs])

In [13]:
# 두칸 위에있는거랑 완벽하게 같음

retriever = database.as_retriever(search_kwargs={'k':3})
retrieved_docs = retriever.invoke(query)

# 3. 답변 생성

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')

In [8]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면

from langchain_upstage import ChatUpstage

llm = ChatUpstage(model = 'solar-pro2',
                  reasoning_effort='high') # 느리지만 더 깊게 추론함 (low, medium)

In [9]:
prompt = f'''[identity]
-당신은 최고의 한국 소득세법 전문가입니다
-[context]를 참고해서 사용자의 질문에 답변해주세요
-[context]는 다음과 같습니다
{retrieved_doc}
-질문 : {query}'''

In [10]:
ai_message = llm.invoke(prompt)

In [11]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 다음과 같은 단계를 거칩니다. 제공된 법조항과 일반적인 세법을 기반으로 계산하며, 자녀 등 추가 공제요건은 없다고 가정합니다.

---

### **1. 근로소득 공제 (근로소득공제)**
- **총급여액**: 5,000만원  
- **공제 계산**:  
  - 1,500만원 이하: 80% 공제 → **1,200만원**  
  - 1,500만원 초과 ~ 4,500만원 이하: 1,200만원 + (4,500만원 - 1,500만원) × 50% = **2,450만원**  
  - 4,500만원 초과: 2,450만원 + (5,000만원 - 4,500만원) × 15% = **2,450만원 + 75만원 = 2,525만원**  
- **공제 상한**: 2,000만원 (법 제47조 제1항 단서)  
  → **근로소득공제액: 2,000만원**

---

### **2. 과세표준 계산**
- **과세표준** = 총급여액 - 근로소득공제  
  = 5,000만원 - 2,000만원 = **3,000만원**

---

### **3. 종합소득 산출세액 계산**
- **세율 적용** (2024년 기준 종합소득세율):  
  - 1,200만원 이하: 6% → 1,200만원 × 6% = **72만원**  
  - 1,200만원 초과 ~ 4,600만원 이하: 15% → (3,000만원 - 1,200만원) × 15% = **270만원**  
  - **산출세액 합계**: 72만원 + 270만원 = **342만원**

---

### **4. 근로소득 세액공제 (근로소득세액공제)**
- **총급여액**: 5,000만원 (3,300만원 초과 ~ 7,000만원 이하 구간)  
- **공제액 계산**:  
  = 74만원 - [(5,000만원 - 3,300만원) × 8/1,000]  
  = 74만원 - (1,700만원 × 0.008) = 74만원 - 13.6만원 = **60.4만원**  
- **최소 공제액**: 66만원 (법 제59조 제2항 제2호)  
  

# 4. langchain으로 답변 생성

In [13]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage
from dotenv import load_dotenv
load_dotenv()

# 1. LLM과 임베딩 초기화
# llm = ChatOpenAI(model='gpt-4.1-mini')
llm = ChatUpstage(model='solar-pro2')
embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

# 2. vertor store load
index_name = 'tax-index-upstage'
vectorstore = PineconeVectorStore(embedding=embedding, 
                               index_name=index_name)

# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type='similarity',
                                     search_kwargs={'k':4})

# 4. 프롬프트 템플릿
template = f'당신은 최고의 한국 소득세 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
prompt = ChatPromptTemplate.from_template(template)

# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

In [14]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {'query':'~'} → '~'
rag_chain = ({'context':retriever | format_documents, 'query':RunnablePassthrough()} # 질문 그대로 전달
             | prompt # prompt에 context와 query 변수 주입
             | llm    # llm에 prompt invoke
             | StrOutputParser()) 

# 7. 실행
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득공제, 세율, 세액공제를 종합적으로 계산해야 하며, 제공된 문맥만으로는 정확한 금액을 산출할 수 없습니다. 추가 정보(가족 수, 자녀 수, 기타 공제 항목 등)가 필요합니다.  \n\n(답변: 제공된 문맥만으로는 정확한 소득세 계산 불가)'